<a href="https://colab.research.google.com/github/kky-ai/tech-demo/blob/main/asking_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Asking Questions Model - Example Usage

##### Download model

In [ ]:
# Downlaod the AQ model from the GDrive
!gdown 1PSaBDLs5z6OF2ej8O_JemzVxNwSaHiic && unzip -u aq.zip


##### Install libs

In [ ]:
!pip install transformers==4.18 sentencepiece

##### Initialize the model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from random import choice


class AQModel:
    """ Asking Questions Model """
    
    def __init__(self):
        self.tokenizer = None
        self.load_tokenizer()
        self.model = None
        self.load_model()
        
    def load_tokenizer(self):
        self.tokenizer = T5Tokenizer('aq/tokenizer/sentencepiece.model')
        print(f'Tokenizer loaded.')

    def load_model(self):
        self.model = T5ForConditionalGeneration.from_pretrained('aq/model', from_tf=True)
        print(f'AQ-Model loaded.')

    def generate(self, context, choose_from=1):
        input_ids = self.tokenizer([context], return_tensors="pt").input_ids
        outputs = self.model.generate(input_ids, 
                                        max_length=128,
                                        num_beams=5,
                                        no_repeat_ngram_size=2, 
                                        num_return_sequences=choose_from, 
                                        early_stopping=True)

        decoded = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        if choose_from > 1:
            print(f'Randomly choosing from: ')
            for d in decoded:
                print(f'{self.parse(d)}')
        
        return self.parse(choice(decoded))
    
    def parse(self, generated):
        try:
            q, a = generated.split('Answer:')
            return q.replace('Question:', '').split('?')[0].strip()+'?', a.strip()
        except:
            print('W: Not parsed.')
            return generated
        

aq = AQModel()

##### Example usage

In [6]:
context = """
    His name is Joe Franklin, and his program ran for 43 years on local 
    television in New York. And he claims that he invented the talk show format.
"""

question, answer = aq.generate(context, choose_from=1)

print(f'Q: {question}')
print(f'A: {answer}')

Q: How long did Joe Franklin's program run on local television?
A: 43 years


##### Sliding window for generating multiple qustions for a given text (txt file)

In [ ]:
!pip install nltk

In [ ]:
import nltk
from tqdm import tqdm

nltk.download('punkt')

def suck_around(context, idx, maxLen):
    words = nltk.tokenize.word_tokenize(context)
    if len(words) <= maxLen:
        return context

    sentences = nltk.sent_tokenize(context)
    cumul = 0
    ret = []
    for sent_i, sent in enumerate(sentences):
        cumul += len(sent)
        
        if idx < cumul:
            # add previous sentence (if any)
            if sent_i > 0:
                ret.append(sentences[sent_i-1])
            
            # add current
            ret.append(sent)
            
            # add following (if any)
            if sent_i < len(sentences)-1:
                ret.append(sentences[sent_i+1])
            
            # keep adding previous (full) sentences until maxLen not exceeded
            tokens_now = nltk.tokenize.word_tokenize(' '.join(ret))
            tokens_in_sent = {si: nltk.tokenize.word_tokenize(s) for si, s in enumerate(sentences[:sent_i-1])}
            
            for si, s in tokens_in_sent.items():
                if len(tokens_now) + len(s) <= maxLen:
                    ret.insert(0, sentences[si])
                else:
                    break
            
            break
    
    return ' '.join(ret)
    
MAX_CONTEXT_LEN = 32
QPERLINE = 1

FN_IN = f'test.txt'
FN_OUT = f'context-question-answer.txt'

with open(FN_OUT, 'w') as tfw:
    with open(FN_IN, 'r') as fr: 
        for line in tqdm(fr.readlines()):
            line = line.strip()
            if line:
                if QPERLINE > 1:
                    for i in range(QPERLINE):
                        idx = int((len(line)/QPERLINE)*i)
                        context = suck_around(line, idx, MAX_CONTEXT_LEN)
                        question, answer = aq.generate(context, choose_from=1)
                        print(context, file=tfw)
                        print(question, answer, sep='\t', file=tfw, end='\n\n')
                else:
                    question, answer = aq.generate(line, choose_from=1)
                    print(line, file=tfw)
                    print(question, answer, sep='\t', file=tfw, end='\n\n')